# Call dialogflow with python api
## precondition
you have to download your json key, and ***export GOOGLE_APPLICATION_CREDENTIALS="/mnt/d/code/sabala/weather-e6aad-0371e7c946bc.json"***


In [ ]:
# check firewall
!rm index.html*
!wget www.google.com

# openai

In [37]:
import os
import openai

openai.api_key = ""

response = openai.Completion.create(
    # model="text-davinci-002",
    model="text-curie-001",
    # prompt="The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.\n\nHuman: Hello, who are you?\nAI: I am an AI created by OpenAI. How can I help you today?\nHuman: I'd like to cancel my subscription.\nAI:",
    prompt="Human: Prova de expor ó vou tirar tudo \nAI:",
    temperature=0.5,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.6,
    stop=[" Human:", " AI:"]
)

# print(response)
output = response['choices'][0]['text'].strip()

# from langdetect import detect
# print(detect("Me fale sobre a lenda da Vitória regi;"))
# print(detect(output))

# print(response['choices'][1]['text'])
# from spacy_langdetect import LanguageDetector
# import spacy
# nlp = spacy.load('en')  # 1
# nlp.add_pipe(LanguageDetector(), name='language_detector', last=True) #2
# text_content = "Er lebt mit seinen Eltern und seiner Schwester in Berlin."
# doc = nlp(text_content) #3
# detect_language = doc._.language #4
# print(detect_language, text_content)
print(output)


To prove you can take it all off, I'll take it all off.


In [ ]:
import requests

def translate(text, src_lang, tar_lang):
    request = requests.post(
        url = "http://main.trans.picoshadow.com:8080/picotrans/translate-internal",
            headers = {
                "Application-ID": "picotransadmin",
                "Manufacturer-ID": "picoshadow",
                "Source-Language-Code": src_lang,
                "Target-Language-Code": tar_lang
            },
            data = text
        )
    # response = request.json()
    # print(response)
    print(request)


translate("maco", "pt-br", "en-us")

In [2]:
import uuid
from google.cloud import dialogflow

session_client = dialogflow.SessionsClient()

# session format: 'projects/*/locations/*/agent/environments/*/users/*/sessions/*'.
def get_session(project_id, session_id, env=None):
    """
    Using the same `session_id` between requests allows continuation
    of the conversation.
    :return: session is a str
    """
    if env is None:
        session = session_client.session_path(project_id, session_id)
        return session
    else:
        assert isinstance(env, str)
        return f"projects/{project_id}/agent/environments/{env}/users/-/sessions/{session_id}"
    

# [START dialogflow_detect_intent_text]
def detect_intent_texts(project_id, session_id, text, language_code, env=None, query_params=None):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    session= get_session(project_id, session_id, env=env)
    print("Session path: {}\n".format(session))

    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    request = dialogflow.DetectIntentRequest(
        session=session, query_input=query_input, query_params=query_params
    )
    response = session_client.detect_intent(request=request)

    # response = session_client.detect_intent(
    #     request={"session": session, "query_input": query_input, "query_parameters": params}
    # )   

    return response


In [3]:
# set credentials, this is a must

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="/mnt/d/code/sabala/mega-sabala-9ibe-940e7527ac9b.json"

## Text to Speech

In [4]:
import os
from google.cloud import texttospeech

# Instantiates a client
client = texttospeech.TextToSpeechClient()
print("=======================")
print('Credendtials from environ: {}'.format(
    os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))
# Set the text input to be synthesized
synthesis_input = texttospeech.SynthesisInput(text="Hello, World!")

# Build the voice request, select the language code ("en-US") and the ssml
# voice gender ("neutral")
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
)

# Select the type of audio file you want returned
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

# Perform the text-to-speech request on the text input with the selected
# voice parameters and audio file type
response = client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)

# The response's audio_content is binary.
with open("output.mp3", "wb") as out:
    # Write the response to the output file.
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')


Credendtials from environ: /mnt/d/code/sabala/mega-sabala-9ibe-940e7527ac9b.json
Audio content written to file "output.mp3"


In [8]:
import requests

url = "http://15.228.123.29:8000/v1/translate/googletranslate"

payload = {"target": "en", "text": "coma"}
headers = {}

response = requests.request("GET", url, headers=headers, params=payload)

print(response.text)


{"data":{"googletranslate_result":"with the"},"code":200,"msg":"ok"}


In [ ]:
import os


def translate_text_with_model(target, text, model="nmt"):
    """Translates text into the target language.

    Make sure your project is allowlisted.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    from google.cloud import translate_v2 as translate
    from google.cloud import translate_v3 as translate

    translate_client = translate.Client()
    translate.Translation

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target, model=model)

    print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}".format(result["translatedText"]))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))


print('Credendtials from environ: {}'.format(
    os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))
translate_text_with_model("en", "你好")


## examples of making session url

In [4]:
from google.cloud import dialogflow

project_id = "sabala-348110"

session_id = str(uuid.uuid4())
# texts = ["Me toque uma música", "toca nos 80", "Tocar música clássica", "parar música"][-1:]
language_code = "en-US"

session = get_session(project_id, session_id, env='new')
session


'projects/sabala-348110/agent/environments/new/users/-/sessions/aefd0fee-dda9-4006-bcdd-ea1f3cc08e57'

# visit Mega Agent and print

In [5]:
project_id = "mega-sabala-9ibe"

session_id = str(uuid.uuid4())
session_id = "dd98a099-bec8-418b-b33f-e1dfd0f1f1d7"

texts = ["how's the weather today", "Você pode me dizer a maneira mais fácil de ganhar dinheiro?"]
texts = ["Sim pode desligar", "Toque a música", "ache Crystal Clear de Hayley Williams"]
language_code = "pt-BR"


In [8]:
from google.cloud import dialogflow_v2
env='test'

context = dialogflow.Context(
    name=f"projects/mega-sabala-9ibe/agent/environments/{env}/users/-/sessions/{session_id}/contexts/sabala-348110.music-switch")
context.lifespan_count = 1
context1 = dialogflow.Context(
    name=f"projects/mega-sabala-9ibe/agent/environments/{env}/users/-/sessions/{session_id}/contexts/__mega_agent_context__")
# context1.lifespan_count = 2147483647
query_params = dialogflow.QueryParameters(contexts=[context, context1])
# query_params = None

text = texts[0]
text = 'Onde Calcinha Preta a banda Calcinha Preta'
response = detect_intent_texts(
        project_id, session_id, text, language_code, env=env, query_params=query_params
)


response.output_audio = "None"
print("=" * 20)
print("Query text: {}".format(response.query_result.query_text))
print(
    "Detected intent: {} (confidence: {})\n".format(
        response.query_result.intent.display_name,
        response.query_result.intent_detection_confidence,
    )
)
print("Fulfillment text: {}\n".format(response.query_result.fulfillment_text))
print("Fulfillment Full: \n{}\n".format(response))
# print(type(response))


Session path: projects/mega-sabala-9ibe/agent/environments/test/users/-/sessions/dd98a099-bec8-418b-b33f-e1dfd0f1f1d7

Query text: Onde Calcinha Preta a banda Calcinha Preta
Detected intent: Chat_53 (confidence: 0.6047602295875549)

Fulfillment text: 

Fulfillment Full: 
response_id: "b0297cea-f1d6-4db3-a8b4-5173bd69b268-1257bac9"
query_result {
  query_text: "Onde Calcinha Preta a banda Calcinha Preta"
  parameters {
  }
  all_required_params_present: true
  fulfillment_messages {
    text {
      text: ""
    }
  }
  output_contexts {
    name: "projects/mega-sabala-9ibe/agent/environments/test/users/-/sessions/dd98a099-bec8-418b-b33f-e1dfd0f1f1d7/contexts/__mega_agent_context__"
    lifespan_count: 2147483647
    parameters {
      fields {
        key: "__most_recent_agent_ids__"
        value {
          list_value {
            values {
              string_value: "b6375f62-e882-4ea6-97be-6d134d0785ce"
            }
          }
        }
      }
    }
  }
  intent {
    name: "pr

In [8]:
from google.cloud import dialogflow_v2


context = dialogflow.Context(
    name=f"projects/mega-sabala-9ibe/agent/environments/{env}/users/-/sessions/{session_id}/contexts/sabala-348110.music-switch")
context1 = dialogflow.Context(
    name=f"projects/mega-sabala-9ibe/agent/environments/{env}/users/-/sessions/{session_id}/contexts/__mega_agent_context__")
query_params = dialogflow.QueryParameters(contexts=[context, context1])


# Initialize request argument(s)
context = dialogflow_v2.Context()
context.name = f"projects/mega-sabala-9ibe/agent/environments/{env}/users/-/sessions/{session_id}/contexts/sabala-348110.music-switch"
# context.name = "sabala-348110.music-switch"

request = dialogflow_v2.CreateContextRequest(   
    parent=f"projects/mega-sabala-9ibe/agent/environments/{env}/users/-/sessions/{session_id}",
    context=context,
)
    
# Create a client
client = dialogflow_v2.ContextsClient()

# Make the request
# response = client.get_context(request=request)
response = client.create_context(request=request)

# Handle the response
print(type(response), response)

<class 'google.cloud.dialogflow_v2.types.context.Context'> name: "projects/mega-sabala-9ibe/agent/environments/test/users/-/sessions/dd98a099-bec8-418b-b33f-e1dfd0f1f1d7/contexts/sabala-348110.music-switch"



# visit media/Music Agent

## visit the default one: draft

In [8]:
project_id = "sabala-348110"

session_id = str(uuid.uuid4())
texts = ["Me toque uma música", "toca nos 80", "Tocar música clássica", "parar música"]
language_code = "en-US"

response = detect_intent_texts(
        project_id, session_id, texts[-1], language_code, env='before_tf'
)

Session path: projects/sabala-348110/agent/environments/before_tf/sessions/d5da4a02-3901-42d4-8ec8-83afcd92b8e7



In [9]:
print( [(_, type(_)) for _ in dir(response) if not _.startswith("_")] )
response.output_audio = "None"
print(response.query_result.intent)

[('output_audio', <class 'str'>), ('output_audio_config', <class 'str'>), ('query_result', <class 'str'>), ('response_id', <class 'str'>), ('webhook_status', <class 'str'>)]
name: "projects/sabala-348110/agent/intents/4bde314e-8feb-4ad1-b1fc-3fe451a7b431"
display_name: "PlayMusic"
end_interaction: true



## visit active version in a specified environment

In [10]:
response = detect_intent_texts(
        project_id, session_id, texts[-1], language_code, env='before_tf'
)

Session path: projects/sabala-348110/agent/environments/before_tf/sessions/d5da4a02-3901-42d4-8ec8-83afcd92b8e7



In [11]:
print(response)

response_id: "663746f1-5693-4599-b3ed-9163c8a5925b-3d49d937"
query_result {
  query_text: "parar m\303\272sica"
  parameters {
    fields {
      key: "artist"
      value {
        string_value: ""
      }
    }
    fields {
      key: "genre"
      value {
        string_value: ""
      }
    }
    fields {
      key: "title"
      value {
        string_value: ""
      }
    }
  }
  all_required_params_present: true
  fulfillment_text: "Agora tocando Nem Amantes Nem Amigos, Trio Parada Dura"
  fulfillment_messages {
    text {
      text: "Agora tocando Nem Amantes Nem Amigos, Trio Parada Dura"
    }
  }
  webhook_payload {
    fields {
      key: "google"
      value {
        struct_value {
          fields {
            key: "expectUserResponse"
            value {
              bool_value: true
            }
          }
          fields {
            key: "richResponse"
            value {
              struct_value {
                fields {
                  key: "items"
     

# Webhook handle
* https://github.com/googleapis/python-dialogflow/blob/HEAD/samples/snippets/webhook.py

In [ ]:
# TODO: change the default Entry Point text to handleWebhook

def handleWebhook(request):

    req = request.get_json()

    responseText = ""
    intent = req["queryResult"]["intent"]["displayName"]

    if intent == "Default Welcome Intent":
        responseText = "Hello from a GCF Webhook"
    elif intent == "get-agent-name":
        responseText = "My name is Flowhook"
    else:
        responseText = f"There are no fulfillment responses defined for Intent {intent}"

    # You can also use the google.cloud.dialogflowcx_v3.types.WebhookRequest protos instead of manually writing the json object
    res = {"fulfillmentMessages": [{"text": {"text": [responseText]}}]}

    return res

In [ ]:
handleWebhook(response)